<a href="https://colab.research.google.com/github/butchland/fastai_xla_extensions/blob/master/samples/test_fast_tpu_pets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 133.6MB 30kB/s 
     |████████████████████████████████| 61kB 3.3MB/s 


In [ ]:
# !pip install -Uqq git+https://github.com/fastai/fastai.git 
!pip install -Uqq fastai --upgrade

     |████████████████████████████████| 194kB 6.9MB/s 
     |████████████████████████████████| 61kB 4.3MB/s 


In [ ]:
!pip install -Uqq git+https://github.com/butchland/fastai_xla_extensions.git

In [ ]:
!pip install -Uqq git+https://github.com/butchland/my_timesaver_utils.git

In [ ]:
!curl -s https://course19.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [ ]:
!pip freeze | grep torch
!pip freeze | grep fast
!pip freeze | grep timesaver

torch==1.7.0+cu101
torch-xla==1.7
torchsummary==1.5.1
torchtext==0.3.1
torchvision==0.8.1+cu101
fastai==2.2.5
fastai-xla-extensions==0.0.7
fastcore==1.3.19
fastdtw==0.3.4
fastprogress==1.0.0
fastrlock==0.5
my-timesaver-utils==0.0.2


Start of kernel restart

In [ ]:
from fastai.vision.all import *
from fastai_xla_extensions.all import *

In [ ]:
# path = untar_data(URLs.MNIST_TINY)
# path = untar_data(URLs.MNIST); Path.BASE_PATH = path; path.ls()
path = untar_data(URLs.PETS)/'images'; Path.BASE_PATH = path; path.ls()

(#7393) [Path('saint_bernard_118.jpg'),Path('saint_bernard_31.jpg'),Path('Bombay_188.jpg'),Path('japanese_chin_82.jpg'),Path('Sphynx_90.jpg'),Path('Persian_262.jpg'),Path('pug_139.jpg'),Path('keeshond_120.jpg'),Path('keeshond_146.jpg'),Path('english_setter_78.jpg')...]

In [ ]:
bs = 64
arch = resnet34

In [ ]:
pat = r'(.+)_\d+.jpg'
data = DataBlock(
    blocks=(ImageBlock,CategoryBlock),
    get_items=get_image_files,
    get_y=using_attr(RegexLabeller(pat),'name'),
    splitter=RandomSplitter(seed=42),
    item_tfms=Resize(28),
    batch_tfms=[]
)

In [ ]:
dls = data.dataloaders(path, bs=bs)

In [ ]:
learner = cnn_learner(dls, arch, metrics=accuracy, concat_pool=False)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


In [ ]:
%%time
learner.xla_fit(5, lr=4e-3)

start fit


epoch,train_loss,valid_loss,accuracy,time


epoch,train_loss,valid_loss,accuracy,time
0,1.158578,3.540231,0.091216,00:22
1,1.588118,3.393301,0.119595,00:11
2,1.997672,3.367104,0.139865,00:06
3,2.288664,3.264080,0.174324,00:06
4,2.475451,3.187805,0.176351,00:06


CPU times: user 721 ms, sys: 310 ms, total: 1.03 s
Wall time: 1min 5s


In [ ]:
learner.validate()

(#2) [3.215650796890259,0.18876861035823822]

In [ ]:
data2 = data.new(item_tfms=Resize(224), batch_tfms=[])

In [ ]:
dls2 = data2.dataloaders(path, bs=64)

In [ ]:
learner.dls = dls2 # replace with size 224 sized

In [ ]:
learner.xla_fit(5, lr=6e-3)

start fit


epoch,train_loss,valid_loss,accuracy,time


epoch,train_loss,valid_loss,accuracy,time
0,0.295006,1.632916,0.514865,01:16
1,0.271668,0.567906,0.808784,00:50
2,0.276961,0.393388,0.885135,00:39
3,0.262727,0.307514,0.905405,00:39
4,0.254267,0.373638,0.878378,00:39
